In [1]:
import numpy as np
import pandas as pd
import openturns as ot
from matplotlib import cm
from matplotlib import pyplot as plt
from bancs import BANCS, NAISAlgorithm, NAISResult

In [2]:
from matplotlib import rc, rcParams, style
#style.use('default')
rcParams['text.usetex'] = True
#rcParams['text.latex.unicode'] = True
rc('font', **{'family': 'Times'})
rc('text', usetex=True)
rc('font', size=16)# Set the default text font size
rc('axes', titlesize=20)# Set the axes title font size
rc('axes', labelsize=16)# Set the axes labels font size
rc('xtick', labelsize=14)# Set the font size for x tick labels
rc('ytick', labelsize=16)# Set the font size for y tick labels
rc('legend', fontsize=16)# Set the legend font size

## Reliability problem #3 from the benchmark proposed by J.M. Bourinet

In [3]:
# PROBLEM 4 with spike
#Zc = "60 * acosh((2*h)/d)"
#Beta   = "2*pi*(freq/3e8)"
#gamma  = alp + 1i*Beta
#
#I1 = "cosh(gamma*L) * (Z0*Zc + ZL*Zc) + sinh(gamma*L) * (Zc^2 + Z0*ZL)"    
#I2 = "sin(Beta*h * cos(ThetaP)) / (Beta*h * cos(ThetaP))"
#I3 = 1i*Beta*cos(ThetaP) * (-sin(ThetaE)*cos(ThetaP)*sin(PhiP) + cos(ThetaE)*cos(PhiP))
#I4 = 0.5 * (Zc + Z0) * ( (exp((gamma + 1i*Beta*sin(ThetaP)*sin(PhiP))*L)-1) / (gamma + 1i*Beta*sin(ThetaP)*sin(PhiP)) )
#I5 = -0.5 * (Zc - Z0) * ( (exp(-(gamma - 1i*Beta*sin(ThetaP)*sin(PhiP))*L)-1) / (gamma - 1i*Beta*sin(ThetaP)*sin(PhiP)) )
#I6 = sin(ThetaE)*sin(ThetaP) * ( Zc - (Zc*cosh(gamma*L) + Z0*sinh(gamma*L)) * exp(1i*Beta*L*sin(ThetaP)*sin(PhiP)) )
#
#g_string = f"abs( (2*{h}*{aE} / {I1}) * {I2} * ( {I3} * ({I4} + {I5}) + {I6}) )"
#g = ot.SymbolicFunction(["L", "h", "d", "ZL", "Z0", "aE", "ThetaE", "ThetaP", "PhiP", "freq", "alp"])

In [4]:
omegap = "((kp / mp) ^ 0.5)"
omegas = "((ks / ms) ^ 0.5)"
gamma = "(ms / mp)"
zetaa = "((zetap + zetas) / 2)"
omegaa = f"(({omegap} + {omegas}) / 2)"
theta = f"(({omegap} - {omegas}) / {omegaa})"
Exs2 = f"(pi_ * S0 / (4 * zetas * {omegas} ^ 3) * ({zetaa} * zetas / (zetap * zetas * (4 * {zetaa}^2 + {theta}^2) + {gamma} * {zetaa}^2) * ((zetap * {omegap}^3 + zetas * {omegas}^3) * {omegap} / (4 * {zetaa} * {omegaa}^4))))"
g_string = f"Fs - ks * 3 * {Exs2}^0.5"
g = ot.SymbolicFunction(["mp", "ms", "kp", "ks", "zetap", "zetas", "Fs", "S0"], [g_string])

In [5]:
mp =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   1.5,     0.1))
ms =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   0.01,    0.1))
kp =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   1.,      0.2))
ks =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   0.01,    0.2))
zetap = ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.05,    0.4))
zetas = ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.02,    0.5))
Fs =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   27.5,    0.1))
S0 =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(   100,     0.1))

In [6]:
X = ot.ComposedDistribution([mp,ms,kp,ks,zetap,zetas,Fs,S0])
Y = ot.CompositeRandomVector(g, ot.RandomVector(X))
threshold = 0.
failure_event = ot.ThresholdEvent(Y, ot.LessOrEqual(), threshold)
# Reference computed using CMC with subset size N=1e7
pf_ref = 3.78 * 1e-7

## BANCS : Bernstein Adaptive Nonparametric Conditional Sampling

In [7]:
N = int(1e4)

In [8]:
ss = ot.SubsetSampling(failure_event)
ss.setMaximumOuterSampling(N)
ss.setMaximumCoefficientOfVariation(-1.0)
ss.setConditionalProbability(0.1)
ss.setBlockSize(1)
timer = ot.TimerCallback(120)
ss.setStopCallback(timer)
ss.run()
res = ss.getResult()
nb_samples = res.getOuterSampling()
pf = res.getProbabilityEstimate()
print(f"Proba SS = {pf:.2e}")
print(f"Relative error = {np.abs(pf - pf_ref) / pf_ref:.2%}")

WRN - The maximum coefficient of variation was set. It won't be used as termination criteria.


Proba SS = 5.16e-07
Relative error = 36.39%


In [9]:
#bancs = BANCS(failure_event, N=N, M=3, p0=0.1, lower_truncatures=[0.] * 8)
#quantiles = bancs.run()
#pf = bancs.compute_pf()
#print("Quantiles =", quantiles)
#print(f"Proba BANCS = {pf:.2e}")
#print(f"Relative error = {np.abs(pf - pf_ref) / pf_ref:.2%}")

In [10]:
nais = NAISAlgorithm(event=failure_event, n_IS=N, rho_quantile=0.1, lower_truncatures=[0.] * 8)
nais.run()
NAIS_result = nais.getResult()
pf = NAIS_result.getProbabilityEstimate()
print(f"Proba BANCS = {pf:.2e}")
print(f"Relative error = {np.abs(pf - pf_ref) / pf_ref:.2%}")

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[None, None, None, None, None, None, None, None]
